In [1]:
from helpers import *

path = "data/ner_annotations.json"
sentences, all_labels, label_list, label_dict = load_data(path)

In [2]:
%env OPENAI_API_KEY=""

env: OPENAI_API_KEY=""


In [3]:
import openai
import os
openai.organization = "org-KvmMsEbe5NcCxonSirXVjXmm"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
all_exemplars = [
    str('\n'.join(list(set(l)))) + "\n\n" + tag_exemplar(e, l)
    for (e, l) in zip(sentences, all_labels)
    ]

In [ ]:
responses = []
log = {
    "n_responses": 0,
    "completion_tokens": 0,
    "prompt_tokens": 0,
    "total_tokens": 0,
    "errors" : []
    }

In [5]:
seed_class = "apparatus-property-type"
class_exemplars = [
    (sentence, labels)
    for sentence, labels in zip(sentences, all_labels)
    if seed_class in labels
    ]

In [6]:
exemplar1_id = 5
exemplar2_id = 10

labels1 = class_exemplars[exemplar1_id][1]
exemplar1 = tag_exemplar(class_exemplars[exemplar1_id][0], labels1)
labels2 = class_exemplars[exemplar2_id][1]
exemplar2 = tag_exemplar(class_exemplars[exemplar2_id][0], labels2)

context_prompt = create_context_prompt(
    exemplar1,
    labels1,
    exemplar2,
    labels2,
    )
print(context_prompt)

You are given this set of entities:
meta
condition-unit
unspecified-material
brand
apparatus-unit
operation
apparatus-property-type
number
material-descriptor
synthesis-apparatus
precursor

The entities are tagged in the examples in an XML-style.

Example 1: Then , the <meta> electrical wire explosion </meta> ( <synthesis-apparatus> NTi - mini P </synthesis-apparatus> , <brand> Nano Tech </brand> , Korea ) was <operation> conducted </operation> at a <apparatus-property-type> feeding distance </apparatus-property-type> of <number> 40 </number> <apparatus-unit> mm </apparatus-unit> and a charge voltage of <number> 320 </number> <condition-unit> V </condition-unit> in the <precursor> Si </precursor> <material-descriptor> nanoparticles - dispersed suspension </material-descriptor>.

Example 2: The <unspecified-material> mixtures </unspecified-material> were then <operation> transferred </operation> to <synthesis-apparatus> cylindrical PVC molds </synthesis-apparatus> ( <apparatus-property-

In [ ]:
context = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : context_prompt}],
    max_tokens=1000,
    temperature=0.7,
    n=1
    )
print(context)

In [ ]:
system_prompt = "Your job is to assist with data augmentation in the material synthesis domain."
augmentation_prompt = "Produce new sentences that are similar to the given examples. Tag all entities in an XML-style."

In [ ]:
try:
    augmentations = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role" : "user", "content" : context_prompt},
            {"role" : "assistant", "content" : context},
            {"role" : "system", "content" : system_prompt},
            {"role" : "user", "content" : augmentation_prompt},
            ],
        max_tokens=1000,
        temperature=0.7,
        n=10
        )
    print(augmentations)
    responses.append(augmentations)
    log["n_responses"] += 1
    log["completion_tokens"] += augmentations["usage"]["completion_tokens"]
    log["prompt_tokens"] += augmentations["usage"]["prompt_tokens"]
    log["total_tokens"] += augmentations["usage"]["total_tokens"]
except Exception as e:
    print("Request failed with Exception:", e)
    log["errors"].append(e)

In [ ]:
import torch
with open("./responses.pt", 'wb') as f:
    torch.save(responses, f)

with open("./log.pt", 'wb') as f:
    torch.save(log, f)